In [1]:
import numpy as np
import pandas as pd
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

# Data Inspection

## Data in customer_feedbacks/

In [117]:
sc = SparkContext('local')
spark = SparkSession(sc)
parquetFile = spark.read.parquet("../data/customer_feedbacks/part-00000-985ad763-a6d6-4ead-a6dd-c02279e9eeba-c000.snappy.parquet")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local) created by __init__ at <ipython-input-2-68232241316b>:1 

In [87]:
parquetFile.printSchema()

root
 |-- ERGEBNISSATZ_ID: decimal(38,10) (nullable = true)
 |-- DATUM_ID: timestamp (nullable = true)
 |-- JAHR_ID: decimal(18,0) (nullable = true)
 |-- MONAT_ID: decimal(18,0) (nullable = true)
 |-- KW_ID: decimal(18,0) (nullable = true)
 |-- TAG_ID: decimal(18,0) (nullable = true)
 |-- KEY_ACCOUNT_ID: string (nullable = true)
 |-- UMFRAGE_ID: decimal(38,10) (nullable = true)
 |-- UMFRAGETYP_ID: decimal(38,10) (nullable = true)
 |-- UMFRAGE_KATEGORIE_ID: decimal(38,10) (nullable = true)
 |-- LAND_ID: decimal(38,10) (nullable = true)
 |-- LABOR_ID: decimal(18,0) (nullable = true)
 |-- CLIENT_ID: decimal(18,0) (nullable = true)
 |-- CLIENT_VERSION_ID: decimal(18,0) (nullable = true)
 |-- COMPLETION_ID: decimal(18,0) (nullable = true)
 |-- NPS_STATUS_ID: decimal(18,0) (nullable = true)
 |-- NPS_KLASSE_ID: decimal(18,0) (nullable = true)
 |-- DKON_KONSUMENT_CODE: string (nullable = true)
 |-- KONTAKTWUNSCH_ID: decimal(18,0) (nullable = true)
 |-- ERSTBESTELLER_ID: decimal(18,0) (nullable

In [113]:
parquetFile.createOrReplaceTempView("parquetFile")

In [114]:
len(parquetFile.columns)

101

In [116]:
spark.sql("SELECT * FROM parquetFile LIMIT 1").show()

+------------------+-------------------+-------+--------+-----+------+--------------+--------------------+-------------+--------------------+------------+--------+---------+-----------------+-------------+-------------+-------------+-------------------+----------------+----------------+-------------+----------+----------------+----------------+------------+-------+---------------+------------+--------+---+------------+--------+------------+----------+---------------+---------+----------------+-----------------+-----------------+----------------+----------------------------+----------------+---------------+------------------+--------------------+----------------------+----------------------+------------------+------------------------+------+--------+-------------+-----------+--------+----------------+--------------+----------------+-----------------+--------------------+--------------------+---------------------+-------------------------+-----------------+----------------------+--------

## Data in customer_feedbacks_cat/

In [45]:
parquetFile2 = spark.read.parquet("../data/customer_feedbacks_cat/part-00000-4820af87-4b19-4958-a7a6-7ed03b76f1b1-c000.snappy.parquet")

In [74]:
parquetFile2.createOrReplaceTempView("parquetFile2")
spark.sql("SELECT * FROM parquetFile2 LIMIT 5").show()

+--------------------+-----------+----------+--------------+--------------+-----------+--------------+
|         KATEGORIE_2|KATEGORIE_1|  STIMMUNG|KATEGORIE_2_ID|KATEGORIE_1_ID|STIMMUNG_ID|KATEGORIE_2_CC|
+--------------------+-----------+----------+--------------+--------------+-----------+--------------+
| Versand - Vorschlag|    Versand|Suggestion|            38|            10|          4|          1038|
|       Versand - Lob|    Versand|    Praise|            39|            10|          2|          1039|
|     Versand - Keine|    Versand|      None|            40|            10|          1|          1040|
|Versand - Beschwerde|    Versand| Complaint|            41|            10|          3|          1041|
|  Editor - Vorschlag|     Editor|Suggestion|            42|            11|          4|          1142|
+--------------------+-----------+----------+--------------+--------------+-----------+--------------+



# Generate dataframe

In [104]:
df=spark.sql("""
SELECT
    T0.KATEGORIE_2     AS CATEGORY_2,
    T0.KATEGORIE_1     AS CATEGORY_1,
    T0.STIMMUNG          AS SENTIMENT,
    T1.ERGEBNISSATZ_ID AS RESPONSE_ID,
    T1.DATUM_ID        AS DATE,
    T1.ANTWORT_WERT    AS TEXT
FROM
    parquetFile2 T0,
    parquetFile T1
WHERE
    T0.KATEGORIE_1_ID = T1.KATEGORIE_1_ID
    AND T0.KATEGORIE_2_ID = T1.KATEGORIE_2_ID
    AND T0.STIMMUNG_ID = T1.STIMMUNG_ID            
    AND (NOT T1.ANTWORT_WERT IS NULL
        AND (T1.UMFRAGE_KATEGORIE_ID = 1
            AND (T1.GRUPPE_ID = 170
                OR T1.GRUPPE_ID = 171)))
""")

In [105]:
df.show()

+--------------------+--------------------+---------+------------------+-------------------+--------------------+
|          CATEGORY_2|          CATEGORY_1|SENTIMENT|       RESPONSE_ID|               DATE|                TEXT|
+--------------------+--------------------+---------+------------------+-------------------+--------------------+
|Delivery & POS Fe...|Delivery & POS Fe...|     None|7492033.0000000000|2012-04-14 00:00:00|Habe bis heute me...|
|  Other - Beschwerde|               Other|Complaint|7492033.0000000000|2012-04-14 00:00:00|Habe bis heute me...|
|                 N/A|                 N/A|      N/A|7492033.0000000000|2012-04-14 00:00:00|  Bin stinke sauer! |
|HPS, IPS & App - ...|      HPS, IPS & App|     None|7509285.0000000000|2012-04-16 00:00:00|Teuer, aber gut. ...|
|Customer Service ...|Customer Service ...|   Praise|7509285.0000000000|2012-04-16 00:00:00|Teuer, aber gut. ...|
|Product Quality -...|     Product Quality|   Praise|7510696.0000000000|2012-04-16 00:00

In [124]:
df.head()

Row(CATEGORY_2='Delivery & POS Feedback - Keine', CATEGORY_1='Delivery & POS Feedback', SENTIMENT='None', RESPONSE_ID=Decimal('7492033.0000000000'), DATE=datetime.datetime(2012, 4, 14, 0, 0), TEXT='Habe bis heute mein Fotobuch noch nicht obwohl ich es im März bestellt habe. War ein Geburtstagsgeschenk doch dieser ist mittlerweile auch vorbei. Keiner weiß wo mein Buch geblieben ist. Schade oder- Vertrauen dahin. Werde ich nie wieder über Müller machen! ')

In [90]:
df.printSchema()

root
 |-- CATEGORY_2: string (nullable = true)
 |-- CATEGORY_1: string (nullable = true)
 |-- SENTIMENT: string (nullable = true)
 |-- RESPONSE_ID: decimal(38,10) (nullable = true)
 |-- DATE: timestamp (nullable = true)
 |-- TEXT: string (nullable = true)



# Data Cleaning

[Row(CATEGORY_2='Delivery & POS Feedback - Keine', CATEGORY_1='Delivery & POS Feedback', SENTIMENT='None', RESPONSE_ID=Decimal('7492033.0000000000'), DATE=datetime.datetime(2012, 4, 14, 0, 0), TEXT='Habe bis heute mein Fotobuch noch nicht obwohl ich es im März bestellt habe. War ein Geburtstagsgeschenk doch dieser ist mittlerweile auch vorbei. Keiner weiß wo mein Buch geblieben ist. Schade oder- Vertrauen dahin. Werde ich nie wieder über Müller machen! '),
 Row(CATEGORY_2='Other - Beschwerde', CATEGORY_1='Other', SENTIMENT='Complaint', RESPONSE_ID=Decimal('7492033.0000000000'), DATE=datetime.datetime(2012, 4, 14, 0, 0), TEXT='Habe bis heute mein Fotobuch noch nicht obwohl ich es im März bestellt habe. War ein Geburtstagsgeschenk doch dieser ist mittlerweile auch vorbei. Keiner weiß wo mein Buch geblieben ist. Schade oder- Vertrauen dahin. Werde ich nie wieder über Müller machen! '),
 Row(CATEGORY_2='N/A', CATEGORY_1='N/A', SENTIMENT='N/A', RESPONSE_ID=Decimal('7492033.0000000000'), DAT